In [4]:
import datetime
import random
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class Flight:
    def __init__(self, flight_number, origin, destination, departure_time, total_seats, base_prices):
        if departure_time <= datetime.datetime.now():
            raise ValueError("Departure time must be in the future.")
        self.flight_number = flight_number
        self.origin = origin
        self.destination = destination
        self.departure_time = departure_time
        self.total_seats = total_seats
        self.available_seats = {'Economy': total_seats // 2, 'Business': total_seats // 3, 'First Class': total_seats // 6}
        self.base_prices = base_prices

    def calculate_dynamic_price(self, seat_class):
        time_remaining = (self.departure_time - datetime.datetime.now()).days
        demand_factor = random.uniform(1.0, 2.0)
        availability_factor = (self.total_seats - sum(self.available_seats.values())) / self.total_seats
        competitor_factor = random.uniform(0.9, 1.1)

        base_price = self.base_prices[seat_class]

        price = base_price * (1 + availability_factor) * demand_factor * competitor_factor

        if time_remaining < 7:
            price *= 1.5
        elif time_remaining < 30:
            price *= 1.2

        return round(price, 2)

    def book_ticket(self, seat_class):
        if seat_class not in self.available_seats:
            raise ValueError(f"Invalid seat class: {seat_class}")

        if self.available_seats[seat_class] > 0:
            ticket_price = self.calculate_dynamic_price(seat_class)
            self.available_seats[seat_class] -= 1
            logging.info(f"Ticket booked: Flight {self.flight_number}, Class {seat_class}, Price ${ticket_price}")
            return f"Ticket booked in {seat_class}! Price: ${ticket_price}"
        else:
            logging.warning(f"No seats available: Flight {self.flight_number}, Class {seat_class}")
            return f"No seats available in {seat_class} class!"

    def flight_info(self):
        return (f"Flight {self.flight_number} | {self.origin} → {self.destination} | "
                f"Departure: {self.departure_time.strftime('%Y-%m-%d %H:%M')} | "
                f"Seats Left: Economy={self.available_seats['Economy']}, "
                f"Business={self.available_seats['Business']}, "
                f"First Class={self.available_seats['First Class']}")

def search_flights(flights, origin, destination, departure_date):
    results = []
    for flight in flights:
        if (flight.origin == origin and flight.destination == destination and
                flight.departure_time.date() == departure_date.date()):
            results.append(flight)
    return results

def get_user_input(prompt, input_type=str):
    while True:
        try:
            user_input = input_type(input(prompt))
            return user_input
        except ValueError:
            print(f"Invalid input. Please enter a valid {input_type.__name__}.")

In [5]:
flight1 = Flight(
    "AI101", "New York", "London", datetime.datetime(2025, 6, 15, 14, 30),
    total_seats=120,
    base_prices={'Economy': 500, 'Business': 1000, 'First Class': 2000}
)

economy_price = flight1.calculate_dynamic_price("Economy")
business_price = flight1.calculate_dynamic_price("Business")
first_class_price = flight1.calculate_dynamic_price("First Class")

print(f"Economy Price: ${economy_price}")
print(f"Business Price: ${business_price}")
print(f"First Class Price: ${first_class_price}")

Economy Price: $838.0
Business Price: $1404.51
First Class Price: $3313.03


In [6]:
print(flight1.book_ticket("Economy"))
print(flight1.flight_info())

2025-03-10 11:58:25,913 - INFO - Ticket booked: Flight AI101, Class Economy, Price $1025.01


Ticket booked in Economy! Price: $1025.01
Flight AI101 | New York → London | Departure: 2025-06-15 14:30 | Seats Left: Economy=59, Business=40, First Class=20


In [7]:
flight2 = Flight(
    "BA202", "London", "Paris", datetime.datetime(2025, 12, 25, 10, 0),
    total_seats=80,
    base_prices={'Economy': 1, 'Business': 2, 'First Class': 3}
)

print(f"Economy Price (Christmas Day): ${flight2.calculate_dynamic_price('Economy')}")

Economy Price (Christmas Day): $1.2


In [8]:
pip install qrcode

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install Pillow

^C
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB 330.3 kB/s eta 0:00:08
   ---------------------------------------- 0.0/2.6 MB 262.6 kB/s eta 0:00:10
    --------------------------------------- 0.0/2.6 MB 326.8 kB/s eta 0:00:08
    --------------------------------------- 0.1/2.6 MB 297.7 kB/s eta 0:00:09
   - -------------------------------------- 0.1/2.6 MB 374.1 kB/s eta 0:00:07
   - -------------------------------------- 0.1/2.6 MB 374.1 kB/s eta 0:00:07
   - -------------------------------------- 0.1/2.6 MB 374.1 kB/s eta 0:00:07
   - -------------------------------------- 0.1/2.6 MB 297.7 kB/s eta 0:00:09
   - -------------------------------------- 0.1/2.6 MB 277.4 kB/s eta 0:00:10
   -- ------------------------------------- 0.1/2.6 MB 293.9 kB/s eta 0:00:09
   -- -------

In [11]:
import qrcode
import io
from PIL import Image  # For displaying the QR code in Jupyter

def generate_upi_qr(upi_id, payee_name, amount):
    """Generates a UPI QR code."""
    upi_string = f"upi://pay?pa={upi_id}&pn={payee_name}&am={amount}"

    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(upi_string)
    qr.make(fit=True)

    img = qr.make_image(fill_color="black", back_color="white")

    return img

def display_qr(qr_image):
    """Displays the QR code image in Jupyter."""
    img_buffer = io.BytesIO()
    qr_image.save(img_buffer, 'PNG')
    img_buffer.seek(0)
    img = Image.open(img_buffer)
    img.show() #this may not work in all environments.
    return img #return the image so it can be shown in some environments

# Example usage within Jupyter
upi_id = "deepamk725@okhdfcbank"  # Replace with your actual UPI ID
payee_name = "AirlineTicket"
amount = 1.0  # Example ticket price (replace with your dynamic price)

qr_image = generate_upi_qr(upi_id, payee_name, amount)
image = display_qr(qr_image) #display the image.